# Steel Plate Faults - Ensemble

In [9]:
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler




# Dataset Loading

In [10]:
data = pd.read_csv('faults.csv')
data.head()

,X_Minimum,X_Maximum,Y_Minimum,Y_Maximum,Pixels_Areas,X_Perimeter,Y_Perimeter,Sum_of_Luminosity,Minimum_of_Luminosity,Maximum_of_Luminosity,...,Orientation_Index,Luminosity_Index,SigmoidOfAreas,Pastry,Z_Scratch,K_Scatch,Stains,Dirtiness,Bumps,Other_Faults
0,42,50,270900,270944,267,17,44,24220,76,108,...,0.8182,-0.2913,0.5822,1,0,0,0,0,0,0
1,645,651,2538079,2538108,108,10,30,11397,84,123,...,0.7931,-0.1756,0.2984,1,0,0,0,0,0,0
2,829,835,1553913,1553931,71,8,19,7972,99,125,...,0.6667,-0.1228,0.2150,1,0,0,0,0,0,0
3,853,860,369370,369415,176,13,45,18996,99,126,...,0.8444,-0.1568,0.5212,1,0,0,0,0,0,0
4,1289,1306,498078,498335,2409,60,260,246930,37,126,...,0.9338,-0.1992,1.0000,1,0,0,0,0,0,0


In [11]:
data.drop('TypeOfSteel_A400', axis = 1)
features = data.values
labels = features[:,27:34]
features = features[:,0:27]

In [12]:
labels = [np.argmax(row) for row in labels]

x_train1, x_test, y_train1, y_test = train_test_split(features, labels,test_size=0.30,random_state=42)

x_train, x_val, y_train, y_val = train_test_split(x_train1, y_train1,test_size=0.30,random_state=17)

# SVM Model Training with MinMax Scaler and Accuracy

In [13]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
x_train1_scaled = scaler.fit_transform(x_train1)
x_test_scaled = scaler.transform(x_test)

svm_model_linear = SVC(kernel='linear', C=1).fit(x_train1_scaled, y_train1)
svm_predictions = svm_model_linear.predict(x_test) 
  
# model accuracy for X_test   
accuracy = svm_model_linear.score(x_test, y_test)
accuracy

0.04631217838765009

# Decision Tree Classifier

In [14]:
model1 = DecisionTreeClassifier(criterion="entropy")
model1.fit(x_train, y_train)
val_pred1=model1.predict(x_val)
test_pred1=model1.predict(x_test)
val_pred1=pd.DataFrame(val_pred1)
test_pred1=pd.DataFrame(test_pred1)



# KNeighbors Classifier

In [15]:
model2 = KNeighborsClassifier(n_neighbors=1)
model2.fit(x_train,y_train)
val_pred2=model2.predict(x_val)
test_pred2=model2.predict(x_test)
val_pred2=pd.DataFrame(val_pred2)
test_pred2=pd.DataFrame(test_pred2)

# Combine Predictions

In [16]:
x_val=pd.DataFrame(x_val)
x_test=pd.DataFrame(x_test)

df_val=pd.concat([x_val, val_pred1,val_pred2],axis=1)
df_test=pd.concat([x_test, test_pred1,test_pred2],axis=1)


# Logistic Regression Ensemble

In [17]:
ensemble_model = LogisticRegression()
ensemble_model.fit(df_val, y_val)

ensemble_accuracy = ensemble_model.score(df_test, y_test)
print("Ensemble Model Accuracy:", ensemble_accuracy)

Ensemble Model Accuracy: 0.4922813036020583


C:\Users\Pranesh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
